In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../Data/requests.csv', index_col='unique_key')
pd.set_option("display.max_columns", 100)
df.head()

,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,status,community_board,borough,open_data_channel_type,park_facility_name,park_borough,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,city,landmark,bbl,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,location,resolution_description,resolution_action_updated_date
unique_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
67351762,2025-12-31 23:59:28,2026-01-01 00:40:32,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,11 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,10029.0,50 EAST 104 STREET,EAST 104 STREET,MADISON AVENUE,PARK AVENUE,MADISON AVENUE,PARK AVENUE,NEW YORK,EAST 104 STREET,1.016080e+09,998068.0,227879.0,40.792141,-73.950097,"{'type': 'Point', 'coordinates': [-73.95009707...",The New York City Police Department responded ...,2026-01-01T00:40:35.000
67344624,2025-12-31 23:59:23,2026-01-01 01:03:42,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,09 MANHATTAN,MANHATTAN,ONLINE,Unspecified,MANHATTAN,10031.0,549 WEST 144 STREET,WEST 144 STREET,AMSTERDAM AVENUE,BROADWAY,AMSTERDAM AVENUE,BROADWAY,NEW YORK,WEST 144 STREET,1.020760e+09,998241.0,239901.0,40.825137,-73.949447,"{'type': 'Point', 'coordinates': [-73.94944723...",The New York City Police Department responded ...,2026-01-01T01:03:44.000
67346873,2025-12-31 23:59:21,2026-01-01 00:58:13,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,Closed,18 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,11234.0,1467 EAST 56 STREET,EAST 56 STREET,AVENUE M,AVENUE N,AVENUE M,AVENUE N,BROOKLYN,EAST 56 STREET,3.078810e+09,1006135.0,165167.0,40.619995,-73.921167,"{'type': 'Point', 'coordinates': [-73.92116739...",The New York City Police Department responded ...,2026-01-01T00:58:18.000
67353004,2025-12-31 23:59:20,2026-01-01 00:57:31,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,15 BROOKLYN,BROOKLYN,ONLINE,Unspecified,BROOKLYN,11235.0,2416 EAST 14 STREET,EAST 14 STREET,AVENUE X,AVENUE Y,AVENUE X,AVENUE Y,BROOKLYN,EAST 14 STREET,3.074150e+09,996476.0,154794.0,40.591542,-73.955979,"{'type': 'Point', 'coordinates': [-73.95597913...",The New York City Police Department responded ...,2026-01-01T00:57:34.000
67350526,2025-12-31 23:59:12,2026-01-01 00:41:16,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,Closed,09 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,10031.0,1635 AMSTERDAM AVENUE,AMSTERDAM AVENUE,WEST 140 STREET,WEST 141 STREET,WEST 140 STREET,WEST 141 STREET,NEW YORK,AMSTERDAM AVENUE,1.020570e+09,998220.0,238974.0,40.822593,-73.949525,"{'type': 'Point', 'coordinates': [-73.94952504...",The Police Department reviewed your complaint ...,2026-01-01T00:41:20.000


In [3]:
# Drop location column since its a duplicate of longitude and latitude
df = df.drop('location', axis = 1)



def conv_to_days(waittime: pd.Timedelta) -> float | None:  # must be a timedelta type
    """Convert wait times to days

    Args:
        waittime (pd.Timedelta): Time difference between closing and created date

    Returns:
        float: If the waittime is a valid time delta, then return the time in days
        None: Returned when waittime is NaT
    """
    try:
        days = abs(waittime.components[0])
        hours = waittime.components[1] / 24
        mins = waittime.components[2] / 60 / 24
        secs = waittime.components[3] / 60 / 60 / 24
        if waittime.components[0] < 0:
            return (days + hours + mins + secs) * -1
        else:
            return days + hours + mins + secs
    except:
        # if NaT, return None
        pass
    
    
df["created_date"] = pd.to_datetime(df["created_date"])
df["closed_date"] = pd.to_datetime(df["closed_date"])
df["waittime"] = df["closed_date"] - df["created_date"]
df["waittime"] = df["waittime"].map(conv_to_days)

In [4]:
# Keep rows with non-negative wait times only
df = df[df['waittime'] >= 0]
df.shape

(599932, 30)

In [5]:
df["waittime"].describe(percentiles=[0.99, 0.999, 0.9999, 0.99999, 0.999999])

count       599932.000000
mean             0.098853
std              0.201573
min              0.000069
99%              0.650317
99.9%            1.664995
99.99%           5.279911
99.999%          9.076572
99.9999%        45.767052
max             87.477442
Name: waittime, dtype: float64

In [6]:
print(f"{df.shape[0]} Rows, {df.shape[1]} Columns")
na_counts = df.isna().sum()
print(f"{'Column':<35} {'NA Counts':<12} {'NA %':<5}")
for col in na_counts.index:
    if na_counts[col] == 0: continue
    print(f"{col:<35} {na_counts[col]:<12} {na_counts[col]/df.shape[0]:.2%}")

599932 Rows, 30 Columns
Column                              NA Counts    NA % 
descriptor                          18335        3.06%
park_facility_name                  851          0.14%
incident_zip                        2960         0.49%
incident_address                    2951         0.49%
street_name                         2953         0.49%
cross_street_1                      4105         0.68%
cross_street_2                      3708         0.62%
intersection_street_1               4105         0.68%
intersection_street_2               3708         0.62%
city                                26816        4.47%
landmark                            26822        4.47%
bbl                                 47422        7.90%
x_coordinate_state_plane            2947         0.49%
y_coordinate_state_plane            2946         0.49%
latitude                            2947         0.49%
longitude                           2947         0.49%
resolution_description              1    

In [7]:
CAT_COLS = [
    "agency",
    "agency_name",
    "complaint_type",
    "descriptor",
    "location_type",
    "community_board",
    "borough",
    "open_data_channel_type",
    "park_facility_name",
    "park_borough",
    # "incident_address",
    # "street_name",
    # "cross_street_1",
    # "cross_street_2",
    # "intersection_street_1",
    # "intersection_street_2",
    "city",
    # "landmark",
]

for i, col in enumerate(CAT_COLS):
    print(f"Processing {col}... ({i+1}/{len(CAT_COLS)})")
    val_cts = df[col].value_counts()
    unique, counts = val_cts.index, val_cts.values

    for i in range(len(unique)):
        key, count = unique[i], counts[i]
        new_key = key.upper() if count > 100 or key.upper() == "UNSPECIFIED" else "OTHER"
        df.loc[df[col] == key, col] = new_key
print()

for col in CAT_COLS:
    val_cts = df[col].value_counts()
    unique, counts = val_cts.index, val_cts.values
    if len(unique) > 10: continue

    print(f"Column: {col}")
    print(f"{'Value':<35} {'Count':<7} {'Percentage %':<15}")
    for i in range(len(unique)):
        key, count = unique[i], counts[i]
        print(f"{unique[i]:<35} {counts[i]:<7} {counts[i]/(np.sum(list(counts)).item()):.2%}")

    print()
    print("="*70)

Processing agency... (1/11)
Processing agency_name... (2/11)
Processing complaint_type... (3/11)
Processing descriptor... (4/11)
Processing location_type... (5/11)
Processing community_board... (6/11)
Processing borough... (7/11)
Processing open_data_channel_type... (8/11)
Processing park_facility_name... (9/11)
Processing park_borough... (10/11)
Processing city... (11/11)

Column: agency
Value                               Count   Percentage %   
NYPD                                599932  100.00%

Column: agency_name
Value                               Count   Percentage %   
NEW YORK CITY POLICE DEPARTMENT     599932  100.00%

Column: borough
Value                               Count   Percentage %   
BROOKLYN                            173876  28.98%
BRONX                               161353  26.90%
QUEENS                              158139  26.36%
MANHATTAN                           91407   15.24%
STATEN ISLAND                       15142   2.52%
UNSPECIFIED                     

In [8]:
df["created_date"] = pd.to_datetime(df["created_date"])

df["created_year"] = df["created_date"].dt.year
df["created_month"] = df["created_date"].dt.month
df["created_day"] = df["created_date"].dt.day
df["created_hour"] = df["created_date"].dt.hour
df["created_weekday"] = df["created_date"].dt.weekday

df["closed_date"] = pd.to_datetime(df["closed_date"])

df["closed_year"] = df["closed_date"].dt.year
df["closed_month"] = df["closed_date"].dt.month
df["closed_day"] = df["closed_date"].dt.day
df["closed_hour"] = df["closed_date"].dt.hour
df["closed_weekday"] = df["closed_date"].dt.weekday
df.head()

,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,status,community_board,borough,open_data_channel_type,park_facility_name,park_borough,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,city,landmark,bbl,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,resolution_description,resolution_action_updated_date,waittime,created_year,created_month,created_day,created_hour,created_weekday,closed_year,closed_month,closed_day,closed_hour,closed_weekday
unique_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
67351762,2025-12-31 23:59:28,2026-01-01 00:40:32,NYPD,NEW YORK CITY POLICE DEPARTMENT,NOISE - RESIDENTIAL,LOUD MUSIC/PARTY,RESIDENTIAL BUILDING/HOUSE,Closed,11 MANHATTAN,MANHATTAN,MOBILE,UNSPECIFIED,MANHATTAN,10029.0,50 EAST 104 STREET,EAST 104 STREET,MADISON AVENUE,PARK AVENUE,MADISON AVENUE,PARK AVENUE,NEW YORK,EAST 104 STREET,1.016080e+09,998068.0,227879.0,40.792141,-73.950097,The New York City Police Department responded ...,2026-01-01T00:40:35.000,0.028519,2025,12,31,23,2,2026,1,1,0,3
67344624,2025-12-31 23:59:23,2026-01-01 01:03:42,NYPD,NEW YORK CITY POLICE DEPARTMENT,NOISE - RESIDENTIAL,LOUD MUSIC/PARTY,RESIDENTIAL BUILDING/HOUSE,Closed,09 MANHATTAN,MANHATTAN,ONLINE,UNSPECIFIED,MANHATTAN,10031.0,549 WEST 144 STREET,WEST 144 STREET,AMSTERDAM AVENUE,BROADWAY,AMSTERDAM AVENUE,BROADWAY,NEW YORK,WEST 144 STREET,1.020760e+09,998241.0,239901.0,40.825137,-73.949447,The New York City Police Department responded ...,2026-01-01T01:03:44.000,0.044664,2025,12,31,23,2,2026,1,1,1,3
67346873,2025-12-31 23:59:21,2026-01-01 00:58:13,NYPD,NEW YORK CITY POLICE DEPARTMENT,BLOCKED DRIVEWAY,PARTIAL ACCESS,STREET/SIDEWALK,Closed,18 BROOKLYN,BROOKLYN,PHONE,UNSPECIFIED,BROOKLYN,11234.0,1467 EAST 56 STREET,EAST 56 STREET,AVENUE M,AVENUE N,AVENUE M,AVENUE N,BROOKLYN,EAST 56 STREET,3.078810e+09,1006135.0,165167.0,40.619995,-73.921167,The New York City Police Department responded ...,2026-01-01T00:58:18.000,0.040880,2025,12,31,23,2,2026,1,1,0,3
67353004,2025-12-31 23:59:20,2026-01-01 00:57:31,NYPD,NEW YORK CITY POLICE DEPARTMENT,NOISE - RESIDENTIAL,LOUD MUSIC/PARTY,RESIDENTIAL BUILDING/HOUSE,Closed,15 BROOKLYN,BROOKLYN,ONLINE,UNSPECIFIED,BROOKLYN,11235.0,2416 EAST 14 STREET,EAST 14 STREET,AVENUE X,AVENUE Y,AVENUE X,AVENUE Y,BROOKLYN,EAST 14 STREET,3.074150e+09,996476.0,154794.0,40.591542,-73.955979,The New York City Police Department responded ...,2026-01-01T00:57:34.000,0.040405,2025,12,31,23,2,2026,1,1,0,3
67350526,2025-12-31 23:59:12,2026-01-01 00:41:16,NYPD,NEW YORK CITY POLICE DEPARTMENT,NOISE - COMMERCIAL,LOUD MUSIC/PARTY,STORE/COMMERCIAL,Closed,09 MANHATTAN,MANHATTAN,MOBILE,UNSPECIFIED,MANHATTAN,10031.0,1635 AMSTERDAM AVENUE,AMSTERDAM AVENUE,WEST 140 STREET,WEST 141 STREET,WEST 140 STREET,WEST 141 STREET,NEW YORK,AMSTERDAM AVENUE,1.020570e+09,998220.0,238974.0,40.822593,-73.949525,The Police Department reviewed your complaint ...,2026-01-01T00:41:20.000,0.029213,2025,12,31,23,2,2026,1,1,0,3


In [9]:
df["has_precise_location"] = df["latitude"].notna()

In [182]:
import os
df_clean = df.copy()
base_dir = os.path.abspath(os.getcwd())
data_dir = os.path.abspath(os.path.join(base_dir, "../Data"))

os.makedirs(data_dir, exist_ok=True)

output_path = os.path.join(data_dir, "requests_cleaned.csv")
df_clean.to_csv(output_path, index=False)

print("Saved to:", output_path)

Saved to: /Users/nithin/ACADEMICS/DataScience/DSC413/NYC-311-Analysis/Data/requests_cleaned.csv
